# What to RAG/vectorize (and why)

We retrieval evidence that helps the LLM map TDs → Node-RED nodes and patterns.

**High-value corpora to embed/index:**

* Our corpus of device *Thing Descriptions (TDs)*, because they are the canonical spec of needed transports and affordances. (Top priority.)
* Example Node-RED flows that we already accept as “correct” (gold standard). These teach layout and wiring style.
* Node-RED documentation pages describing node types and their configuration (socket, mqtt, http-in, websocket nodes).
* node-specific docs for third-party nodes we use (e.g., `node-red-contrib-websocket`, `node-red-node-wot`, `node-red-contrib-xxxx`).
* Protocol docs with examples (WebSocket message formats, MQTT topic conventions) if we need message parsing examples or JSON schemas.
* Domain information via web search

**Less useful/low priority:**
* Entire books on Node-RED — useful for background but too noisy unless we want style guidance.

**How to use RAG:**
* Retrieve the most similar TD + best-matching example flow, and pass those into the LLM as context when generating the subflow. This reduces hallucination of transports or incorrect wiring.


In [1]:
urls = [
    "https://nodered.org/docs/tutorials/first-flow",
    "https://nodered.org/docs/tutorials/second-flow",
    "https://cookbook.nodered.org/http/create-an-http-endpoint",
    "https://cookbook.nodered.org/http/handle-query-parameters",
    "https://cookbook.nodered.org/http/handle-url-parameters",
    "https://cookbook.nodered.org/http/access-http-request-headers",
    "https://cookbook.nodered.org/http/include-data-from-another-flow",
    "https://cookbook.nodered.org/http/serve-json-content",
    "https://cookbook.nodered.org/http/simple-get-request",
    "https://cookbook.nodered.org/http/set-request-url",
    "https://cookbook.nodered.org/http/set-request-url-template",
    "https://cookbook.nodered.org/http/set-query-string",
    "https://cookbook.nodered.org/http/parse-json-response",
    "https://cookbook.nodered.org/http/get-binary-response",
    "https://cookbook.nodered.org/http/set-request-header",
    "https://cookbook.nodered.org/mqtt/connect-to-broker",
    "https://cookbook.nodered.org/mqtt/publish-to-topic",
    "https://cookbook.nodered.org/mqtt/set-publish-topic",
    "https://cookbook.nodered.org/mqtt/publish-retained-message",
    "https://cookbook.nodered.org/mqtt/subscribe-to-topic",
    "https://cookbook.nodered.org/mqtt/receive-json",
    "https://nodered.org/docs/developing-flows/flow-structure",
    "https://nodered.org/docs/developing-flows/message-design",
    "https://nodered.org/docs/api/admin/types",
    "https://www.w3.org/TR/wot-architecture/",
    "https://www.w3.org/TR/wot-thing-description11/",
    "https://github.com/eclipse-thingweb/node-red/blob/main/node-red-node-wot/examples/server-side-flows.json",
    "https://github.com/eclipse-thingweb/node-red/blob/main/node-red-node-wot/examples/client-side-flows.json",
    "https://github.com/eclipse-thingweb/node-red/tree/main/node-red-node-wot",
]

In [2]:
%pip install langchain-openai langchain-core langchain langchain-text-splitters langchain-community bs4

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY") or getpass.getpass("Enter OpenAI API Key: ")

In [4]:
# Optional
os.environ["LANGSMITH_API_KEY"] = os.getenv("LANGSMITH_API_KEY") or getpass.getpass("Enter LangSmith API Key")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "LLMSoSComp"

# Indexing

## Load

Load ALL pages at once with WebBaseLoader

In [5]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader(web_paths=urls)
docs = loader.load()

print(len(docs), "documents loaded.")
assert len(docs) == len(urls)

print(docs[0].metadata)
print(docs[0].page_content[:500])

USER_AGENT environment variable not set, consider setting it to identify your requests.


29 documents loaded.
{'source': 'https://nodered.org/docs/tutorials/first-flow', 'title': 'Creating your first flow : Node-RED', 'language': 'en'}






Creating your first flow : Node-RED

















Node-RED

home
about
blog
documentation
forum
flows
github







docs
        
         • tutorials
        • first flow













             




Creating your first flow
Overview
This tutorial introduces the Node-RED editor and creates a flow that demonstrates
the Inject, Debug and Function nodes.
1. Access the editor
With Node-RED running, open the editor in a web browser.
If you are using a browser on the same computer that is 


In [6]:
total_chars = sum(len(doc.page_content) for doc in docs)
print(f"Total Characters: {total_chars}")


Total Characters: 653079


## Split

In [7]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=5000,
    chunk_overlap=500,
    add_start_index=True,  # Attaches the starting character index of each chunk in the original text as metadata.
)

all_splits = text_splitter.split_documents(docs)

print(f"splitted docs into {len(all_splits)} chunks")

splitted docs into 164 chunks


In [8]:
all_splits[5]

Document(metadata={'source': 'https://cookbook.nodered.org/http/handle-url-parameters', 'title': 'Handle url parameters in an HTTP endpoint : Node-RED', 'language': 'en', 'start_index': 5}, page_content='Handle url parameters in an HTTP endpoint : Node-RED\n\n\n\n\n\n\n\n\n\n\nNode-RED\n\nhome\nabout\nblog\ndocumentation\nforum\nflows\ngithub\n\n\n\n\n\n\n\n\ncookbook\n        \n        \n        • http• url parameters\n\n\n\n\n\nV\n\n\n\n\n\n\n\n        \xa0\n\n\n\n\nHandle url parameters in an HTTP endpoint\nProblem\nYou want to create a single HTTP endpoint that can handle requests where parts\nof the path are set per-request.\nFor example, a single endpoint that can handle requests to both:\nhttp://example.com/hello-param/Nick\nhttp://example.com/hello-param/Dave\n\nSolution\nUse named path parameters in your HTTP In node’s URL\nproperty and then access the specific value provided in a request using the\nmsg.req.params property of the message.\nFlow\n\n[{"id":"ce53954b.31ac68","typ

In [9]:
all_splits[162]

Document(metadata={'source': 'https://github.com/eclipse-thingweb/node-red/tree/main/node-red-node-wot', 'title': 'node-red/node-red-node-wot at main · eclipse-thingweb/node-red · GitHub', 'description': 'Collection of Node-RED packages of Eclipse Thingweb - node-red/node-red-node-wot at main · eclipse-thingweb/node-red', 'language': 'en', 'start_index': 92}, page_content='node-red/node-red-node-wot at main · eclipse-thingweb/node-red · GitHub\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to content\n\n\n\n\n\n\n\n\n\n\n\n\n\nNavigation Menu\n\nToggle navigation\n\n\n\n\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n            Sign in\n          \n\n\n \n\n\nAppearance settings\n\n\n\n\n\n\n\n\n\n\n\n        Platform\n        \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n          GitHub Copilot\n\n        \n\n        Write better code with AI\n      \n\n\n\n\n\n\n\n\n          GitHub Spark\n\n            \n              New\n            \n\n\n        Build 

## Store

In [10]:
# select embedding model
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [11]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)

In [12]:
document_ids = vector_store.add_documents(documents=all_splits)

In [13]:
print(document_ids[:3])

['d5124d68-5c0c-4489-a66e-52b3498c47d3', '347ab6d1-4589-4219-a8fd-ab9c7d039fa2', '912f2d93-d2af-4724-82e3-16bf97715221']


# Retrieval and Generation

In [14]:
import sys
import os

parent_dir = os.path.abspath("..")
sys.path.append(parent_dir)

import utils


In [15]:
from langchain.chat_models import init_chat_model

model = init_chat_model(
    model="gpt-4.1",
    temperature=0.0
)

In [17]:
# Retrieve relevant docs from your vector store
retrieved_docs = vector_store.similarity_search(
    utils.smart_home_system_description, k=4
)

# Join text for prompt context
docs_content = "\n\n".join(
    f"[Source: {doc.metadata.get('source','unknown')}]\n{doc.page_content}"
    for doc in retrieved_docs
)


In [18]:
docs_content

'[Source: https://www.w3.org/TR/wot-architecture/]\nFigure\n          5 Control Agent\n\n\n\n\n\n5.4\n        Remote Access\n\nThis deployment scenario contains a mobile remote\n      controller (e.g., on a smartphone) as shown in Figure 6. The remote controller can switch between\n      different network connections and protocols, e.g., between a\n      cellular network and a home network, which is using protocols\n      such as Wi-Fi and Bluetooth. When the controller is in the\n      home network it is a trusted device and no additional\n      security or access control is required. When it is outside of\n      the trusted network, additional access control and security\n      mechanisms must be applied to ensure a trusted relationship.\n      Note that in this scenario the network connectivity may\n      change due to switching between different network access\n      points or cellular base stations.\nIn this pattern, the remote controller and the electronic\n      appliance have a

In [19]:
user_prompt = """
    Desired Workflow: \n {desired_workflow}
    List of Devices as WoT Thing Descriptions: \n {thing_directory}

    Use the following relevant docs to help generate the workflow: \n {context_docs}
    """

In [20]:
from langchain_core.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate, 
    HumanMessagePromptTemplate
)

prompt_template = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(utils.system_prompt),
    HumanMessagePromptTemplate.from_template(user_prompt)
])

In [21]:
prompt_template.input_variables

['context_docs', 'desired_workflow', 'thing_directory']

In [22]:
pipeline = (
    {
        "thing_directory": lambda x: x["thing_directory"],
        "desired_workflow": lambda x: x["desired_workflow"],
        "context_docs": lambda x: x["context_docs"]
    }
    | prompt_template
    | model
    | {"generated_workflow": lambda x: x.content}
)

In [23]:
response = pipeline.invoke({
    "thing_directory": utils.smart_home_td_full,
    "desired_workflow": utils.smart_home_system_description,
    "context_docs": docs_content
})

In [ ]:
response

{'generated_workflow': '[\n  {\n    "id": "n1-wm-finished-event",\n    "type": "websocket in",\n    "z": "main",\n    "name": "WashingMachine finishedCycle",\n    "server": "",\n    "client": "",\n    "path": "ws://localhost:1880/things/wm/finishedCycle",\n    "x": 120,\n    "y": 80,\n    "wires": [\n      [\n        "n2-blink-leds"\n      ]\n    ]\n  },\n  {\n    "id": "n2-blink-leds",\n    "type": "http request",\n    "z": "main",\n    "name": "Blink LEDs",\n    "method": "POST",\n    "ret": "txt",\n    "url": "http://localhost:1880/things/leds/blink",\n    "headers": {},\n    "x": 350,\n    "y": 80,\n    "wires": [\n      []\n    ]\n  },\n  {\n    "id": "n3-motion-detected",\n    "type": "websocket in",\n    "z": "main",\n    "name": "MotionSensor motionDetected",\n    "server": "",\n    "client": "",\n    "path": "ws://localhost:1880/things/motionsensor/motiondetected",\n    "x": 120,\n    "y": 180,\n    "wires": [\n      [\n        "n4-light-on"\n      ]\n    ]\n  },\n  {\n    "id

In [25]:
# formatting the response

import json

# Parse twice: first to extract string, second to get proper JSON
workflow_str = response['generated_workflow']
workflow = json.loads(workflow_str)

# Now `workflow` is a list of dicts — ready for Node-RED
print(json.dumps(workflow, indent=2))

[
  {
    "id": "n1-wm-finished-event",
    "type": "websocket in",
    "z": "main",
    "name": "WashingMachine finishedCycle",
    "server": "",
    "client": "",
    "path": "ws://localhost:1880/things/wm/finishedCycle",
    "x": 120,
    "y": 80,
    "wires": [
      [
        "n2-blink-leds"
      ]
    ]
  },
  {
    "id": "n2-blink-leds",
    "type": "http request",
    "z": "main",
    "name": "Blink LEDs",
    "method": "POST",
    "ret": "txt",
    "url": "http://localhost:1880/things/leds/blink",
    "headers": {},
    "x": 350,
    "y": 80,
    "wires": [
      []
    ]
  },
  {
    "id": "n3-motion-detected",
    "type": "websocket in",
    "z": "main",
    "name": "MotionSensor motionDetected",
    "server": "",
    "client": "",
    "path": "ws://localhost:1880/things/motionsensor/motiondetected",
    "x": 120,
    "y": 180,
    "wires": [
      [
        "n4-light-on"
      ]
    ]
  },
  {
    "id": "n4-light-on",
    "type": "http request",
    "z": "main",
    "name"